<a href="https://colab.research.google.com/github/cras-lab/ForMe/blob/main/Consistency_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import files
from google.colab import drive
import pandas as pd
import numpy as np
import math
import shutil
import os

USE_FIXED_FILE = False
SHEET_NAME = [  'AI-Ex', 'G-Ex', 'CEO', 'A-CEO', 'ALL' ]
idx_label = [ ' AI_L', 'AI_W','AI_GW', '일반_L', '일반_W', '일반_GW', 'CEO_L', 'CEO_W', 'CEO_GW',  ]
#idx_label_T = [ ' AI_L', 'AI_W','AI_GW', 'RANK', '일반_L', '일반_W', '일반_GW', 'RANK', 'CEO_L', 'CEO_W', 'CEO_GW', 'RANK'  ]

#idx_label = [ ' AI_L', 'AI_W','AI_GW', '일반_L', '일반_W', '일반_GW', 'CEO_L', 'CEO_W', 'CEO_GW', '전체-CEOL', '전체-CEOW', '전체-CEOGW', '전체L', '전체W', '전체GW' ]
idx_label_T = [ ' AI_L', 'AI_W','AI_GW', 'RANK', '일반_L', '일반_W', '일반_GW', 'RANK', 'CEO_L', 'CEO_W', 'CEO_GW', 'RANK', '전체-CEOL', 'A-CEO_W', 'A-CEO_GW', 'RANK', '전체L', '전체W', '전체GW', 'RANK'  ]
#idx_label_T = [ 'AI_W','AI_GW', 'RANK', '일반W', '일반GW', 'RANK', 'CEO_W', 'CEO_GW', 'RANK', '전체-CEOW', '전체-CEOGW', 'RANK', '전체W', '전체GW', 'RANK'  ]

if USE_FIXED_FILE:
  local_file_path = "D:\\Research\\answers.xlsx"
  local_file_path = "C:/nsispromotion_log.txt"
  src_path = "/content/answers.xlsx"
  shutil.copy( local_file_path, src_path )
else:
  uploaded = files.upload()
  filename = list(uploaded.keys())[0]
  src_path = "/content/" + filename

NUM_CLASS = 5
NUM_Q = 4
NUM_COMB = math.comb( NUM_Q, 2)

def GetMatrix( col ):
  matrix = [ [1, col[0], col[1], col[2] ],
             [1/col[0], 1, col[3], col[4]],
             [1/col[1], 1/col[3], 1, col[5]],
             [ 1/col[2], 1/col[4], 1/col[5], 1]
           ]
  return matrix

def FindEigenValue( col ):
  mat = GetMatrix(col)
  eigenvalues, eigenvectors = np.linalg.eig(mat)
  principal_eigenvalue = max(abs(eigenvalues))

  return principal_eigenvalue

def FindEigenValues( col ):
  eigens = np.zeros(NUM_CLASS)

  for i in range( 0, NUM_CLASS):
    col1 = col[i*NUM_COMB:(i+1)*NUM_COMB]
    eigens[i] = FindEigenValue(col1)

  return eigens

# 각 사람별로 가중치를 구함
def GetWeight(col):
  harmonic = np.zeros(NUM_Q*NUM_CLASS)
  weight = np.zeros(NUM_Q*NUM_CLASS)

  for i in range( 0, NUM_CLASS):
    col1 = col[i*NUM_COMB:(i+1)*NUM_COMB]
    mat = GetMatrix(col1)

    for j in range(0, NUM_Q):
      harmonic[i*NUM_Q+j] = (mat[j][0]*mat[j][1]*mat[j][2]*mat[j][3])**(1/NUM_Q)

    total = sum( harmonic[i*NUM_Q:i*NUM_Q+4])

    weight[i*NUM_Q:i*NUM_Q+4 ] = [ x/total for x in harmonic[i*NUM_Q:i*NUM_Q+4]]

  return weight

drive.mount( '/content/drive')
dest_path = '/content/drive/MyDrive/result.xls'
tmp_path = "/content/temp.xls"

Total_df = pd.DataFrame()

# STEP 1 모든 sheet에 대해 데이터를 읽는다.
for num in range(0, len(SHEET_NAME)):
  df = pd.read_excel(src_path,sheet_name=SHEET_NAME[num] )
  num_persons = len(df.columns) - 1  # 가장 앞의 Q 열을 제외

  e_values = np.zeros( (num_persons, NUM_CLASS) )

  CI = np.zeros( (num_persons+4, NUM_CLASS) )  # 마지막 행에 평균, 최대, 최소, 중앙값을 넣는다.
  Local_Wgt = np.zeros( (num_persons+1, NUM_CLASS*NUM_Q ) )  # 제일 마지막 행에 전체 평균을 넣는다.
  Global_Wgt = np.zeros( (num_persons+1, NUM_CLASS*NUM_Q )  ) # 제일 마지막 행에 전체 평균을 넣는다.

  # 단계 2 각자의 CI와 Local 가중치를 계산
  for i in range(0, num_persons):
      col = df[ df.columns[i+1]].values
      e_values[i] = FindEigenValues( col )  # Primary Eigen value 추출
      CI[i] = [ (x-4)/3/0.9 for x in e_values[i] ]
      Local_Wgt[i] = GetWeight(col)

  # 각자의 SERM 가중치를 곱해서 각자의 글로벌 가중치를 구한다.
  for p in range(0, num_persons):
    for k in range(1, 5):
      Global_Wgt[ p, 0:4 ] = Local_Wgt[p, 0:4]    # 리스트의 앞 4개 값이 SERM의 가중치
      Global_Wgt[ p, NUM_Q*k:NUM_Q*k+4 ] = [ x * Global_Wgt[p, k-1] for x in Local_Wgt[ p, NUM_Q*k:NUM_Q*k+4 ] ]

  # 위에서 구한 모든 값을 편의상 개별적인 데이터프레임으로 만든다.
  ev_df = pd.DataFrame( e_values )
  ev_df.columns = [ 'SERM', 'Subject', 'Environment', 'Resource', 'Mechanism']
  ev_df.set_index( df.columns[1:],  inplace=True)

  CI_df = pd.DataFrame( CI )
  CI_df.columns = [ 'SERM', 'Subject', 'Environment', 'Resource', 'Mechanism']
  CI_df.set_index( pd.Index(df.columns[1:].to_list() + ["평균", "최대", "최소", "중앙값"]),  inplace=True)

  Local_Wgt_df = pd.DataFrame( Local_Wgt  )
  Local_Wgt_df.set_index( pd.Index(df.columns[1:].to_list()  + ["평균"]),  inplace=True)
  Local_Wgt_df.columns = [ 'S', 'E', 'R', 'M', 'S1', 'S2', 'S3', 'S4', 'E1', 'E2', 'E3', 'E4', 'R1', 'R2', 'R3', 'R4', 'M1', 'M2', 'M3', 'M4']

  Global_Wgt_df = pd.DataFrame( Global_Wgt )
  Global_Wgt_df.set_index( pd.Index(df.columns[1:].to_list() + ["평균"]),  inplace=True)
  Global_Wgt_df.columns = [ 'S', 'E', 'R', 'M', 'S1', 'S2', 'S3', 'S4', 'E1', 'E2', 'E3', 'E4', 'R1', 'R2', 'R3', 'R4', 'M1', 'M2', 'M3', 'M4']

  Local_Wgt_df.iloc[-1, :] = Local_Wgt_df.iloc[:-1, :].mean()  # 마지막 행 제외한 나머지 평균 계산
  Global_Wgt_df.iloc[-1, :] = Global_Wgt_df.iloc[:-1, :].mean()  # 마지막 행 제외한 나머지 평균 계산

  CI_df.iloc[-4, :] =  CI_df.iloc[:-4, :].mean()  # 마지막 4행 제외한 나머지 평균 계산
  CI_df.iloc[-3, :] =  CI_df.iloc[:-4, :].max()  # 마지막 4행 제외한 나머지 최대 계산
  CI_df.iloc[-2, :] =  CI_df.iloc[:-4, :].min()  # 마지막 4행 제외한 나머지 최소 계산
  CI_df.iloc[-1, :] =  CI_df.iloc[:-4, :].median()  # 마지막 4행 제외한 나머지 중앙값 계산

  Wgt_sorted = pd.DataFrame()   # 가중치에 따라 정렬
  Wgt_original = pd.DataFrame()   # 정렬하지 않은 원래 데이터

  Total_df[SHEET_NAME[num]+"_L"] =  Local_Wgt_df.iloc[-1]
  Total_df[SHEET_NAME[num]+"_G"] =  Global_Wgt_df.iloc[-1]

  for idx in range(0, len(Local_Wgt_df.columns), 4):

    group = Local_Wgt_df.iloc[:, idx:idx+4]
    last_row = group.iloc[-1]
    sorted_group = group.sort_values(by=last_row.name, axis=1, ascending=False)

    Wgt_original = pd.concat( [Wgt_original, group], axis=1)
    Wgt_sorted = pd.concat( [Wgt_sorted, sorted_group], axis=1)


  global_rank = np.zeros( len( Local_Wgt_df.columns )   )

  # SERM 순위
  sorted_indices = np.argsort(Total_df[SHEET_NAME[num]+"_G"][0:4])[::-1]
  global_rank[0:4] = [sorted_indices.tolist().index(i) + 1 for i in range(len(Total_df[SHEET_NAME[num]+"_G"][0:4]))]

  # SERM 가중치를 곱한 요인들 글로벌 순위
  sorted_indices = np.argsort(Total_df[SHEET_NAME[num]+"_G"][4:])[::-1]
  global_rank[4:] = [sorted_indices.tolist().index(i) + 1 for i in range(len(Total_df[SHEET_NAME[num]+"_G"][4:]))]

  Total_df[SHEET_NAME[num]+"순위"] =  global_rank

  if num == 0:  # 처음 호출 , 파일 생성
    with pd.ExcelWriter(tmp_path, engine='openpyxl') as writer:
    #    ev_df.to_excel(writer, sheet_name='Eigen Values', index=True)
      CI_df.to_excel(writer, sheet_name='CI_' + SHEET_NAME[num] , index=True, float_format="%.3f")
      Wgt_sorted.to_excel(writer, sheet_name='가중치_' + SHEET_NAME[num], startrow=1, startcol=0, index=True, float_format="%.3f")
      Global_Wgt_df.to_excel(writer, sheet_name='가중치_' + SHEET_NAME[num], index=True, startrow=len(Wgt_sorted)+4, startcol=0, float_format="%.3f")

      ws = writer.sheets['가중치_' + SHEET_NAME[num]]
      ws.cell(row=1, column=1, value='요인별로 Local 가중치 순으로 정렬')
      ws.cell(row=len(Wgt_sorted)+4, column=1, value='영역 가중치를 반영한 글로벌 가중치 수치')

  else:   # 파일 이미 존재
    with pd.ExcelWriter( tmp_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
  #    ev_df.to_excel(writer, sheet_name='Eigen Values', index=True)
      CI_df.to_excel(writer, sheet_name='CI_' + SHEET_NAME[num] , index=True, float_format="%.3f")
      Wgt_sorted.to_excel(writer, sheet_name='가중치_' + SHEET_NAME[num],startrow=1, startcol=0, index=True, float_format="%.3f"  )
      Global_Wgt_df.to_excel(writer, sheet_name='가중치_' + SHEET_NAME[num], index=True, startrow=len(Wgt_sorted)+4, startcol=0, float_format="%.3f")

      ws = writer.sheets['가중치_' + SHEET_NAME[num]]
      ws.cell(row=1, column=1, value='요인별로 Local 가중치 순으로 정렬')
      ws.cell(row=len(Wgt_sorted)+4, column=1, value='영역 가중치를 반영한 글로벌 가중치 수치')

with pd.ExcelWriter( tmp_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
  Total_df.to_excel(writer, sheet_name='GW_Sorted', index=True, startrow=1, startcol=1, float_format="%.3f")


shutil.copy( tmp_path , dest_path )


Saving answers.xlsx to answers (37).xlsx
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/result.xls'